In [3]:
import numpy as np
import tensorflow as tf

from keras.layers import Dense, Activation
from keras.models import Model
from keras.applications import imagenet_utils
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.activations import relu, sigmoid, softmax

from sklearn.metrics import confusion_matrix

import os
import itertools
import shutil
import random
import matplotlib.pyplot as plt

In [4]:
mobile = tf.keras.applications.mobilenet.MobileNet()
#mobile.summary()

In [5]:
#fetching the model and exclusding the last 6 layers of the mobile net
x = mobile.layers[-6].output
 #store it in a variable x
output = Dense(units=10, activation='softmax')(x)


In [10]:
#construct the reconstructed model
rec_model = Model(inputs=mobile.input, outputs=output) #we have out original mobile net layer
# we have included our layer for the last 5/6 that we ommiteed, therefoer, we have to train 
#this new model, and this we do by, freezing some parts of the previous inorder to maintin 
#the weights

In [ ]:
#experimantally came up with 26 inrder ro have a decently performing model
for layer in rec_model.layers[:23]: #choose the last 23, omit the first (88-23)
    layer.trainable = False #train 23rd to last layer
rec_model.summary()

In [13]:
#Compile the Model for Training

rec_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy')

In [ ]:
#Train the Model by calling the fit function
rec_model.fit()